In [1]:
# pip install transformers
# !pip install -q decord
# pip install youtube-dl ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 12.7 MB/s 
     |████████████████████████████████| 182 kB 44.2 MB/s 
     |████████████████████████████████| 7.6 MB 53.6 MB/s 


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from transformers import VideoMAEFeatureExtractor, VideoMAEForVideoClassification
import torch

feature_extractor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
model = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

c:\Users\kyunomi\anaconda3\envs\lauretta\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


VideoMAEForVideoClassification(
  (videomae): VideoMAEModel(
    (embeddings): VideoMAEEmbeddings(
      (patch_embeddings): VideoMAEPatchEmbeddings(
        (projection): Conv3d(3, 768, kernel_size=(2, 16, 16), stride=(2, 16, 16))
      )
    )
    (encoder): VideoMAEEncoder(
      (layer): ModuleList(
        (0): VideoMAELayer(
          (attention): VideoMAEAttention(
            (attention): VideoMAESelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): VideoMAESelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): VideoMAEIntermediate(
            (dense): Linear(in_f

In [219]:
import youtube_dl

url = 'https://www.youtube.com/shorts/CQZdidYMMPw'
# we use replace as youtube_dl does not currently work with youtube shorts url
url = url.replace('shorts/','watch?v=')
ydl_opts = {
        'format':'mp4[width<=?360][height<=?360]',
        'noplaylist': True,
        'outtmpl': '/content/drive/MyDrive/lauretta/test.mp4',
    }
ydl = youtube_dl.YoutubeDL(ydl_opts)
try:
  os.remove('/content/drive/MyDrive/lauretta/test.mp4')
except:
  pass
finally:
  ydl.download([url])


[youtube] CQZdidYMMPw: Downloading webpage
[youtube] Downloading just video CQZdidYMMPw because of --no-playlist
[download] Destination: /content/drive/MyDrive/lauretta/test.mp4
[download] 100% of 375.46KiB in 00:05


In [221]:
from ipywidgets import Video

# video_path = "/content/drive/MyDrive/lauretta/pizza.mp4" 
# video_path = "/content/drive/MyDrive/lauretta/pull up.mp4" 
# video_path = "/content/drive/MyDrive/lauretta/baby.mp4" 
video_path = "/content/drive/MyDrive/lauretta/test.mp4" 
Video.from_file(video_path, width=240)

Video(value=b'\x00\x00\x00\x18ftypmp42\x00\x00\x00\x00isommp42\x00\x00\x17Dmoov\x00\x00\x00lmvhd\x00\x00\x00\x…

In [230]:
from decord import VideoReader, cpu # decord is a faster alternative of opencv
import numpy as np

# video clip consists of 300 frames (10 seconds at 30 FPS)
vr = VideoReader(video_path, num_threads=1, ctx=cpu(0)) 

def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
  # As the model only allow 16 frames as input
  # we will sample 16 frames from the video

  # get the total number of frames that our sample would span across
  converted_len = int(clip_len * frame_sample_rate)
  # get a randomised max frame of the video. This would be the last frame out of the 16 frame
  end_idx = np.random.randint(converted_len, seg_len)
  # get the first frame sample
  str_idx = end_idx - converted_len
  # evenly space out the frame between the start and end frame according to the frame sample rate
  index = np.linspace(str_idx, end_idx, num=clip_len)
  # ensure frames' index is within range and convert to numpy array
  index = np.clip(index, str_idx, end_idx - 1).astype(np.int64)

  # # we change the linspace to be spread from 20 to 80 percentile to capture more info across the video
  # index = np.linspace(seg_len*0.2, seg_len*0.8, num=clip_len)
  # index = np.clip(index, seg_len*0.2, seg_len*0.8 - 1).astype(np.int64)
  return index

# we run vr.seek(0) to start the read
vr.seek(0)
# get the np array of frames
index = sample_frame_indices(clip_len=16, frame_sample_rate=5, seg_len=len(vr))
# get the frame rgb values as numpy (16 frames, height, width, rgb)
buffer = vr.get_batch(index).asnumpy()
buffer.shape

(16, 360, 202, 3)

In [223]:
# create a list of NumPy arrays
video = [buffer[i] for i in range(buffer.shape[0])]

# extract features based on previous pipeline
encoding = feature_extractor(video, return_tensors="pt")
print(encoding.pixel_values.shape)

torch.Size([1, 16, 3, 224, 224])


# Load Model

In [224]:
pixel_values = encoding.pixel_values.to(device)

# forward pass
with torch.no_grad():
  outputs = model(pixel_values)
  logits = outputs.logits

In [232]:
predicted_class_idx = logits.argmax(-1).item()

print("Predicted class:", model.config.id2label[predicted_class_idx])
     

Predicted class: riding or walking with horse
